#**Shopee App Review Analysis Sentiment - by Evlin Sitanggang**

##Preparation

In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


In [27]:
import re
import string
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

import csv
import requests
import tensorflow as tf
from io import StringIO

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download as nltk_download
nltk_download('punkt')
nltk_download('punkt_tab')
nltk_download('stopwords')
nltk_download('wordnet')

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional, MaxPooling1D, Flatten, GRU, SimpleRNN, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from keras.layers import LeakyReLU, BatchNormalization

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Read dataset
df = pd.read_csv('/content/drive/MyDrive/Dataset/review_shopee.csv')

In [4]:
# Menampilkan lima data teratas.
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ff8384c6-afde-4cba-8f33-c9b91cd37275,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Tempat belanja online sempurna, saya tidak per...",5,157,3.37.31,2024-11-15 10:37:23,"Hai kak Lee Arshi, makasih untuk bintang 5 nya...",2024-11-15 11:39:39,3.37.31
1,7a326ed0-f5da-40f0-a101-172cfaa23ea9,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi belanja pertama dan satu-satunya yang...,5,132,3.37.31,2024-11-15 08:42:56,"Hi kak, thankyou ya buat bintang limanya. Moga...",2024-11-15 09:19:12,3.37.31
2,afdcefb2-1f64-417c-bb6d-229497cb990c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sudah mendingan aplikasi shopee tidak sperti b...,5,847,3.36.20,2024-11-10 08:00:19,"Hi kak, maaf ya udh bikin km kesel. pastiin ap...",2023-06-17 06:15:51,3.36.20
3,bc106341-f27a-472b-a4c1-040ca61bfbd5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya senang dengan aplikasi di Shopee... Belan...,5,231,3.36.20,2024-11-14 01:53:22,"Hii kak Agustine Jemmy, makasih ya buat review...",2024-11-14 02:05:05,3.36.20
4,4da387f4-4140-47e4-9d5f-9394515ad5c2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Jika pengiriman lebih detail sprti pengemudi o...,5,4,3.37.31,2024-11-14 02:34:34,"Hai kak, terima kasih banyak ya kak atas ratin...",2024-11-14 03:27:32,3.37.31


In [5]:
# Menampilkan info dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              30000 non-null  object
 1   userName              30000 non-null  object
 2   userImage             30000 non-null  object
 3   content               30000 non-null  object
 4   score                 30000 non-null  int64 
 5   thumbsUpCount         30000 non-null  int64 
 6   reviewCreatedVersion  27602 non-null  object
 7   at                    30000 non-null  object
 8   replyContent          22857 non-null  object
 9   repliedAt             22857 non-null  object
 10  appVersion            27602 non-null  object
dtypes: int64(2), object(9)
memory usage: 2.5+ MB


In [6]:
# Menghapus missing value pada kolom 'content'
clean_df = df.dropna(subset=['content'])

# Menghapus df duplikat
clean_df = clean_df.drop_duplicates()

In [7]:
# Menampilkan info dataset
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              30000 non-null  object
 1   userName              30000 non-null  object
 2   userImage             30000 non-null  object
 3   content               30000 non-null  object
 4   score                 30000 non-null  int64 
 5   thumbsUpCount         30000 non-null  int64 
 6   reviewCreatedVersion  27602 non-null  object
 7   at                    30000 non-null  object
 8   replyContent          22857 non-null  object
 9   repliedAt             22857 non-null  object
 10  appVersion            27602 non-null  object
dtypes: int64(2), object(9)
memory usage: 2.5+ MB


In [8]:
clean_df = clean_df[['content']]
clean_df = clean_df.rename(columns={"content": "review"})
clean_df

,review
0,"Tempat belanja online sempurna, saya tidak per..."
1,Aplikasi belanja pertama dan satu-satunya yang...
2,Sudah mendingan aplikasi shopee tidak sperti b...
3,Saya senang dengan aplikasi di Shopee... Belan...
4,Jika pengiriman lebih detail sprti pengemudi o...
...,...
29995,"Puas banget pake shopee, bisa buat belanja rut..."
29996,Okelah belanja di shope tapi tipsnya harus ben...
29997,Tadi ny Paling enak blanja di shope krn bny ca...
29998,Sejauh ini sih menggunakan shopee blm ada masa...


In [9]:
# Menampilkan info dataset
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  30000 non-null  object
dtypes: object(1)
memory usage: 234.5+ KB


In [10]:
# Mendefinisikan fungsi-fungsi untuk preprocessing text

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = re.sub(r'[^\w\s]', '', text) # remove numbers
    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text):
    text = text.lower()
    return text

slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Jika input berupa list, gunakan langsung
    if isinstance(text, list):
        stemmed_words = [stemmer.stem(word) for word in text]
    else:
        # Jika input berupa string, pecah menjadi list kata
        words = text.split()
        stemmed_words = [stemmer.stem(word) for word in words]

    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [11]:
clean_df['text_clean'] = clean_df['review'].apply(cleaningText)
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)
clean_df['final_text'] = clean_df['text_stopword'].apply(toSentence)

In [12]:
clean_df

,review,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,final_text
0,"Tempat belanja online sempurna, saya tidak per...",Tempat belanja online sempurna saya tidak pern...,tempat belanja online sempurna saya tidak pern...,tempat belanja online sempurna saya tidak pern...,"[tempat, belanja, online, sempurna, saya, tida...","[belanja, online, sempurna, ganti, update, sho...",belanja online sempurna ganti update shopeepay...
1,Aplikasi belanja pertama dan satu-satunya yang...,Aplikasi belanja pertama dan satusatunya yang ...,aplikasi belanja pertama dan satusatunya yang ...,aplikasi belanja pertama dan satusatunya yang ...,"[aplikasi, belanja, pertama, dan, satusatunya,...","[aplikasi, belanja, satusatunya, menyajikan, p...",aplikasi belanja satusatunya menyajikan piliha...
2,Sudah mendingan aplikasi shopee tidak sperti b...,Sudah mendingan aplikasi shopee tidak sperti b...,sudah mendingan aplikasi shopee tidak sperti b...,sudah mendingan aplikasi shopee tidak sperti b...,"[sudah, mendingan, aplikasi, shopee, tidak, sp...","[mendingan, aplikasi, shopee, sperti, kemarin,...",mendingan aplikasi shopee sperti kemarin yg te...
3,Saya senang dengan aplikasi di Shopee... Belan...,Saya senang dengan aplikasi di Shopee Belanja ...,saya senang dengan aplikasi di shopee belanja ...,saya senang dengan aplikasi di shopee belanja ...,"[saya, senang, dengan, aplikasi, di, shopee, b...","[senang, aplikasi, shopee, belanja, main, game...",senang aplikasi shopee belanja main game perta...
4,Jika pengiriman lebih detail sprti pengemudi o...,Jika pengiriman lebih detail sprti pengemudi o...,jika pengiriman lebih detail sprti pengemudi o...,jika pengiriman lebih detail sprti pengemudi o...,"[jika, pengiriman, lebih, detail, sprti, penge...","[pengiriman, detail, sprti, pengemudi, ojol, e...",pengiriman detail sprti pengemudi ojol enak pa...
...,...,...,...,...,...,...,...
29995,"Puas banget pake shopee, bisa buat belanja rut...",Puas banget pake shopee bisa buat belanja ruti...,puas banget pake shopee bisa buat belanja ruti...,puas banget pake shopee bisa buat belanja ruti...,"[puas, banget, pake, shopee, bisa, buat, belan...","[puas, banget, pake, shopee, belanja, rutinan,...",puas banget pake shopee belanja rutinan krna c...
29996,Okelah belanja di shope tapi tipsnya harus ben...,Okelah belanja di shope tapi tipsnya harus ben...,okelah belanja di shope tapi tipsnya harus ben...,okelah belanja di shope tapi tipsnya harus ben...,"[okelah, belanja, di, shope, tapi, tipsnya, ha...","[okelah, belanja, shope, tipsnya, bener, bener...",okelah belanja shope tipsnya bener bener cari ...
29997,Tadi ny Paling enak blanja di shope krn bny ca...,Tadi ny Paling enak blanja di shope krn bny ca...,tadi ny paling enak blanja di shope krn bny ca...,tadi ny paling enak blanja di shope krn bny ca...,"[tadi, ny, paling, enak, blanja, di, shope, kr...","[ny, enak, blanja, shope, krn, bny, cashback, ...",ny enak blanja shope krn bny cashback gratis o...
29998,Sejauh ini sih menggunakan shopee blm ada masa...,Sejauh ini sih menggunakan shopee blm ada masa...,sejauh ini sih menggunakan shopee blm ada masa...,sejauh ini sih menggunakan shopee blm ada masa...,"[sejauh, ini, sih, menggunakan, shopee, blm, a...","[shopee, blm, shopee, hadiah, voucher, gratis,...",shopee blm shopee hadiah voucher gratis ongkir...


In [13]:
clean_df = clean_df.dropna()
print("Number of Reviews :", clean_df.shape[0])

Number of Reviews : 30000


In [14]:
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
reader = csv.reader(StringIO(response.text), delimiter=',')
for row in reader:
    lexicon_positive[row[0]] = int(row[1])


# Loads negative lexicon data from GitHub
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
reader = csv.reader(StringIO(response.text), delimiter=',')
for row in reader:
    lexicon_negative[row[0]] = int(row[1])

In [15]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text.split():
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]

    for word in text.split():
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]

    polarity=''

    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity

In [16]:
results = clean_df['final_text'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['sentiment_score'] = results[0]
clean_df['sentiment'] = results[1]
print(clean_df['sentiment'].value_counts())

sentiment
positive    18300
negative     9691
neutral      2009
Name: count, dtype: int64


In [17]:
clean_df.to_csv('labeling_data.csv', index=False)

In [18]:
df = clean_df
df

,review,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,final_text,sentiment_score,sentiment
0,"Tempat belanja online sempurna, saya tidak per...",Tempat belanja online sempurna saya tidak pern...,tempat belanja online sempurna saya tidak pern...,tempat belanja online sempurna saya tidak pern...,"[tempat, belanja, online, sempurna, saya, tida...","[belanja, online, sempurna, ganti, update, sho...",belanja online sempurna ganti update shopeepay...,-3,negative
1,Aplikasi belanja pertama dan satu-satunya yang...,Aplikasi belanja pertama dan satusatunya yang ...,aplikasi belanja pertama dan satusatunya yang ...,aplikasi belanja pertama dan satusatunya yang ...,"[aplikasi, belanja, pertama, dan, satusatunya,...","[aplikasi, belanja, satusatunya, menyajikan, p...",aplikasi belanja satusatunya menyajikan piliha...,26,positive
2,Sudah mendingan aplikasi shopee tidak sperti b...,Sudah mendingan aplikasi shopee tidak sperti b...,sudah mendingan aplikasi shopee tidak sperti b...,sudah mendingan aplikasi shopee tidak sperti b...,"[sudah, mendingan, aplikasi, shopee, tidak, sp...","[mendingan, aplikasi, shopee, sperti, kemarin,...",mendingan aplikasi shopee sperti kemarin yg te...,-12,negative
3,Saya senang dengan aplikasi di Shopee... Belan...,Saya senang dengan aplikasi di Shopee Belanja ...,saya senang dengan aplikasi di shopee belanja ...,saya senang dengan aplikasi di shopee belanja ...,"[saya, senang, dengan, aplikasi, di, shopee, b...","[senang, aplikasi, shopee, belanja, main, game...",senang aplikasi shopee belanja main game perta...,-4,negative
4,Jika pengiriman lebih detail sprti pengemudi o...,Jika pengiriman lebih detail sprti pengemudi o...,jika pengiriman lebih detail sprti pengemudi o...,jika pengiriman lebih detail sprti pengemudi o...,"[jika, pengiriman, lebih, detail, sprti, penge...","[pengiriman, detail, sprti, pengemudi, ojol, e...",pengiriman detail sprti pengemudi ojol enak pa...,10,positive
...,...,...,...,...,...,...,...,...,...
29995,"Puas banget pake shopee, bisa buat belanja rut...",Puas banget pake shopee bisa buat belanja ruti...,puas banget pake shopee bisa buat belanja ruti...,puas banget pake shopee bisa buat belanja ruti...,"[puas, banget, pake, shopee, bisa, buat, belan...","[puas, banget, pake, shopee, belanja, rutinan,...",puas banget pake shopee belanja rutinan krna c...,5,positive
29996,Okelah belanja di shope tapi tipsnya harus ben...,Okelah belanja di shope tapi tipsnya harus ben...,okelah belanja di shope tapi tipsnya harus ben...,okelah belanja di shope tapi tipsnya harus ben...,"[okelah, belanja, di, shope, tapi, tipsnya, ha...","[okelah, belanja, shope, tipsnya, bener, bener...",okelah belanja shope tipsnya bener bener cari ...,4,positive
29997,Tadi ny Paling enak blanja di shope krn bny ca...,Tadi ny Paling enak blanja di shope krn bny ca...,tadi ny paling enak blanja di shope krn bny ca...,tadi ny paling enak blanja di shope krn bny ca...,"[tadi, ny, paling, enak, blanja, di, shope, kr...","[ny, enak, blanja, shope, krn, bny, cashback, ...",ny enak blanja shope krn bny cashback gratis o...,-6,negative
29998,Sejauh ini sih menggunakan shopee blm ada masa...,Sejauh ini sih menggunakan shopee blm ada masa...,sejauh ini sih menggunakan shopee blm ada masa...,sejauh ini sih menggunakan shopee blm ada masa...,"[sejauh, ini, sih, menggunakan, shopee, blm, a...","[shopee, blm, shopee, hadiah, voucher, gratis,...",shopee blm shopee hadiah voucher gratis ongkir...,8,positive


In [19]:
X = df['final_text'].values
y = df['sentiment'].values

In [20]:
# Tokenisasi
max_features = 2500
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X)

# Label encoding
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(y)
y = to_categorical(integer_encoded)

# Menentukan jumlah kelas
num_classes = y.shape[1]

In [61]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.94:
            print(f"\nAccuracy has reached beyond 0.94, stopping training at epoch {epoch+1}")
            self.model.stop_training = True
early_stopping = CustomCallback()

##CNN

In [81]:
# Separating training, validation, and test data
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

In [82]:
# Building a CNN model
model_CNN = Sequential()
model_CNN.add(Embedding(max_features, 512, input_length=X.shape[1]))
model_CNN.add(Conv1D(64, kernel_size=5, activation='relu'))
model_CNN.add(MaxPooling1D(pool_size=2))
model_CNN.add(Flatten())
model_CNN.add(Dense(64, activation='relu'))
model_CNN.add(Dropout(0.5))
model_CNN.add(Dense(num_classes, activation='softmax'))
model_CNN.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model with custom early stopping
batch_size = 2056
history_CNN = model_CNN.fit(X_train, y_train, epochs=50, batch_size=batch_size, verbose=2, callbacks=[early_stopping], validation_data=(X_val, y_val))

Epoch 1/50
12/12 - 52s - 4s/step - accuracy: 0.5799 - loss: 0.9118 - val_accuracy: 0.6015 - val_loss: 0.8402
Epoch 2/50
12/12 - 80s - 7s/step - accuracy: 0.6219 - loss: 0.7924 - val_accuracy: 0.7022 - val_loss: 0.7001
Epoch 3/50
12/12 - 53s - 4s/step - accuracy: 0.7600 - loss: 0.6335 - val_accuracy: 0.7967 - val_loss: 0.5657
Epoch 4/50
12/12 - 49s - 4s/step - accuracy: 0.8270 - loss: 0.4894 - val_accuracy: 0.8433 - val_loss: 0.4453
Epoch 5/50
12/12 - 83s - 7s/step - accuracy: 0.8687 - loss: 0.3835 - val_accuracy: 0.8637 - val_loss: 0.3835
Epoch 6/50
12/12 - 81s - 7s/step - accuracy: 0.8925 - loss: 0.3103 - val_accuracy: 0.8733 - val_loss: 0.3414
Epoch 7/50
12/12 - 82s - 7s/step - accuracy: 0.9100 - loss: 0.2665 - val_accuracy: 0.8859 - val_loss: 0.3272
Epoch 8/50
12/12 - 82s - 7s/step - accuracy: 0.9149 - loss: 0.2410 - val_accuracy: 0.8933 - val_loss: 0.3191
Epoch 9/50
12/12 - 82s - 7s/step - accuracy: 0.9228 - loss: 0.2214 - val_accuracy: 0.8937 - val_loss: 0.3094
Epoch 10/50
12/12 -

In [85]:
# Model Evaluation
score_CNN, acc_CNN = model_CNN.evaluate(X_test, y_test, verbose=2)
print("Test accuracy: %.2f" % (acc_CNN))

94/94 - 1s - 14ms/step - accuracy: 0.9150 - loss: 0.3038
Test accuracy: 0.92


In [86]:
# Save Model
model_CNN.save('model_CNN.h5')

##LSTM

In [89]:
# Separating training, validation, and test data
X_train_val, X_testLSTM, y_train_val, y_testLSTM = train_test_split(X, y, test_size=0.1, random_state=42)

X_trainLSTM, X_valLSTM, y_trainLSTM, y_valLSTM = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

In [90]:
# Building a LSTM model
model_LSTM = Sequential()
model_LSTM.add(Embedding(max_features, 256, input_length=X.shape[1]))
model_LSTM.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), return_sequences=True))
model_LSTM.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01)))
model_LSTM.add(Dense(128, activation='relu'))
model_LSTM.add(Dense(64, activation='relu'))
model_LSTM.add(Dense(num_classes, activation='softmax'))
model_LSTM.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model with custom early stopping
batch_size = 64
history_LSTM = model_LSTM.fit(X_trainLSTM, y_trainLSTM, epochs=50, batch_size=batch_size, verbose=2, callbacks=[early_stopping], validation_data=(X_valLSTM, y_valLSTM))

Epoch 1/50
380/380 - 117s - 309ms/step - accuracy: 0.6107 - loss: 1.2654 - val_accuracy: 0.6015 - val_loss: 0.8600
Epoch 2/50
380/380 - 146s - 385ms/step - accuracy: 0.6267 - loss: 0.8260 - val_accuracy: 0.7774 - val_loss: 0.6403
Epoch 3/50
380/380 - 137s - 360ms/step - accuracy: 0.8529 - loss: 0.4390 - val_accuracy: 0.8689 - val_loss: 0.3657
Epoch 4/50
380/380 - 143s - 377ms/step - accuracy: 0.9005 - loss: 0.3106 - val_accuracy: 0.8937 - val_loss: 0.3264
Epoch 5/50
380/380 - 140s - 367ms/step - accuracy: 0.9190 - loss: 0.2627 - val_accuracy: 0.9070 - val_loss: 0.3069
Epoch 6/50
380/380 - 143s - 377ms/step - accuracy: 0.9243 - loss: 0.2436 - val_accuracy: 0.8981 - val_loss: 0.3188
Epoch 7/50
380/380 - 143s - 376ms/step - accuracy: 0.9334 - loss: 0.2222 - val_accuracy: 0.9041 - val_loss: 0.3141
Epoch 8/50
380/380 - 141s - 371ms/step - accuracy: 0.9348 - loss: 0.2151 - val_accuracy: 0.9000 - val_loss: 0.3427
Epoch 9/50
380/380 - 116s - 305ms/step - accuracy: 0.9388 - loss: 0.2026 - val_a

In [91]:
# Model Evaluation
score_LSTM, acc_LSTM = model_LSTM.evaluate(X_testLSTM, y_testLSTM, verbose=2)
print("Test accuracy: %.2f" % (acc_LSTM))

94/94 - 7s - 79ms/step - accuracy: 0.9220 - loss: 0.2748
Test accuracy: 0.92


In [92]:
# Save Model
model_LSTM.save('model_LSTM.h5')

##GRU

In [93]:
# Separating training, validation, and test data
X_train_val, X_testGRU, y_train_val, y_testGRU = train_test_split(X, y, test_size=0.05, random_state=42)

X_trainGRU, X_valGRU, y_trainGRU, y_valGRU = train_test_split(X_train_val, y_train_val, test_size=0.05, random_state=42)

In [94]:
# Building a GRU model
model_GRU = Sequential()
model_GRU.add(Embedding(max_features, 512, input_length=X.shape[1]))
model_GRU.add(SpatialDropout1D(0.3))
model_GRU.add(Bidirectional(GRU(64, return_sequences=True)))
model_GRU.add(Bidirectional(GRU(128, return_sequences=False)))
model_GRU.add(Dense(128, activation='relu',kernel_regularizer=l2(0.01)))
model_GRU.add(Dropout(0.5))
model_GRU.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model_GRU.add(Dropout(0.5))
model_GRU.add(Dense(num_classes, activation='softmax'))
model_GRU.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model with custom early stopping
batch_size = 64
history_GRU = model_GRU.fit(X_trainGRU, y_trainGRU, epochs=20, batch_size=batch_size, verbose=2, validation_data=(X_valGRU, y_valGRU),callbacks=[early_stopping])

Epoch 1/20
424/424 - 278s - 656ms/step - accuracy: 0.8081 - loss: 0.8423 - val_accuracy: 0.8884 - val_loss: 0.3528
Epoch 2/20
424/424 - 321s - 757ms/step - accuracy: 0.9052 - loss: 0.3160 - val_accuracy: 0.9025 - val_loss: 0.3032
Epoch 3/20
424/424 - 266s - 628ms/step - accuracy: 0.9206 - loss: 0.2661 - val_accuracy: 0.8968 - val_loss: 0.3019
Epoch 4/20
424/424 - 326s - 768ms/step - accuracy: 0.9307 - loss: 0.2355 - val_accuracy: 0.9151 - val_loss: 0.2781
Epoch 5/20
424/424 - 272s - 641ms/step - accuracy: 0.9372 - loss: 0.2136 - val_accuracy: 0.9060 - val_loss: 0.3070
Epoch 6/20

Accuracy has reached beyond 0.94, stopping training at epoch 6
424/424 - 321s - 756ms/step - accuracy: 0.9443 - loss: 0.1943 - val_accuracy: 0.9109 - val_loss: 0.3114


In [95]:
# Model Evaluation
score_GRU, acc_GRU = model_GRU.evaluate(X_testGRU, y_testGRU, verbose=2)
print("Test accuracy: %.2f" % (acc_GRU))

47/47 - 5s - 112ms/step - accuracy: 0.9153 - loss: 0.2896
Test accuracy: 0.92


In [96]:
# Save Model
model_GRU.save('model_GRU.h5')

##Evaluation

In [101]:
results_df = pd.DataFrame({
    'Model': ['LSTM', 'CNN', 'GRU'],
    'Accuracy Train': [round(x, 2) for x in [history_LSTM.history['accuracy'][-1], history_CNN.history['accuracy'][-1], history_GRU.history['accuracy'][-1]]],
    'Accuracy Test': [round(x, 2) for x in [acc_LSTM, acc_CNN, acc_GRU]]
})

results_df

,Model,Accuracy Train,Accuracy Test
0,LSTM,0.94,0.92
1,CNN,0.94,0.92
2,GRU,0.94,0.92


##Predicted

In [102]:
# Predict with foreign data
new_texts = [
    "Shopee keren banget banyak promo gratis ongkir dan diskon gede-gedean.",
    "Aplikasi Shopee sangat mengecewakan, sering nge-lag parah saat flash sale. Barang murah jadi percuma karena error terus, bikin pembeli nggak kebagian.",
    "Semoga bisa ditingkatkan lg kualitasnya agar lebih baik."
]

# True Value
true_labels = ["positive", "negative", "neutral"]

# Tokenize new text
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = pad_sequences(new_sequences, maxlen=X.shape[1])

# prediction
predictions_LSTM = model_LSTM.predict(new_padded_sequences)
predictions_CNN = model_CNN.predict(new_padded_sequences)
predictions_GRU = model_GRU.predict(new_padded_sequences)

# change to label
predicted_labels_LSTM = np.argmax(predictions_LSTM, axis=1)
predicted_labels_CNN = np.argmax(predictions_CNN, axis=1)
predicted_labels_GRU = np.argmax(predictions_GRU, axis=1)

predicted_labels_LSTM = labelencoder.inverse_transform(predicted_labels_LSTM)
predicted_labels_CNN = labelencoder.inverse_transform(predicted_labels_CNN)
predicted_labels_GRU = labelencoder.inverse_transform(predicted_labels_GRU)

# Show the prediction result
for text, true_label, label_LSTM, label_CNN, label_GRU in zip(new_texts, true_labels, predicted_labels_LSTM, predicted_labels_CNN, predicted_labels_GRU):
    print(f"Text: {text}\nOriginal Label: {true_label}\nPredicted Label LSTM: {label_LSTM}\nPredicted Label CNN: {label_CNN}\nPredicted Label GRU: {label_GRU}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Text: Shopee keren banget banyak promo gratis ongkir dan diskon gede-gedean.
Original Label: positive
Predicted Label LSTM: positive
Predicted Label CNN: positive
Predicted Label GRU: positive

Text: Aplikasi Shopee sangat mengecewakan, sering nge-lag parah saat flash sale. Barang murah jadi percuma karena error terus, bikin pembeli nggak kebagian.
Original Label: negative
Predicted Label LSTM: negative
Predicted Label CNN: negative
Predicted Label GRU: negative

Text: Semoga bisa ditingkatkan lg kualitasnya agar lebih baik.
Original Label: neutral
Predicted Label LSTM: neutral
Predicted Label CNN: neutral
Predicted Label GRU: neutral

